In [ ]:
import json
import numpy as np

# ✅ 사용할 관절 목록
keypoints = [
    "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle",
    "Neck", "Left Palm", "Right Palm", "Back", "Waist",
    "Left Foot", "Right Foot"
]

# ✅ JSON 데이터 로드 함수 (5개 각도 전처리)
def load_json_skeleton(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    num_frames = len(data["frames"])
    num_joints = len(keypoints)
    num_features = 2  # (x, y)
    num_views = 5     # view1 ~ view5

    # ✅ (1, 프레임, 뷰, 관절, 좌표) 형태로 데이터 배열 생성
    X_data = np.zeros((1, num_frames, num_views, num_joints, num_features), dtype=np.float32)

    views = ["view1", "view2", "view3", "view4", "view5"]

    # ✅ JSON 데이터 -> 배열 변환
    for frame_idx, frame in enumerate(data["frames"]):
        for view_idx, view in enumerate(views):
            pts = frame.get(view, {}).get("pts", {})
            for joint_idx, joint_name in enumerate(keypoints):
                if joint_name in pts:
                    X_data[0, frame_idx, view_idx, joint_idx, 0] = pts[joint_name]["x"]
                    X_data[0, frame_idx, view_idx, joint_idx, 1] = pts[joint_name]["y"]

    return X_data, data.get("type_info", None)

# ✅ 여러 개의 JSON 파일을 한 번에 로드하는 함수 (관절별 라벨 포함)
def load_labeled_json_skeleton(file_paths, labels):
    X_data_list = []
    y_data_list = []

    for file_path, label in zip(file_paths, labels):
        X, _ = load_json_skeleton(file_path)
        X_data_list.append(X)

        # ✅ 관절별 라벨로 변환 (0 = 정상, 1 = 잘못됨)
        if isinstance(label, int):
            # 기존 방식 (0/1)일 경우 모든 관절에 동일한 라벨 적용
            joint_labels = np.full((len(keypoints),), label)
        else:
            # 리스트 형태일 경우 관절별 라벨로 처리
            joint_labels = np.array(label)

        y_data_list.append(joint_labels)

    # ✅ 여러 개의 파일을 하나의 NumPy 배열로 병합
    X_train = np.concatenate(X_data_list, axis=0)   # (batch_size, frames, views, joints, features)
    y_train = np.array(y_data_list)                 # (batch_size, joints)

    return X_train, y_train

# ✅ 올바른 자세와 잘못된 자세 데이터를 함께 로드
file_paths = [
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-2-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-3-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-4-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-5-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-6-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-7-561.json",  # 올바른 자세
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-562.json",  
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-563.json",  
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-564.json",  
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-565.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-566.json", 
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-567.json", 
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_09/Day35_201107_F/D35-1-568.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-2-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-3-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-4-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-5-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-6-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-7-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-562.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-563.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-564.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-565.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-566.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-567.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_08/Day33_201105_F/D33-1-568.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-2-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-3-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-4-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-5-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-6-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-7-561.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-562.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-563.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-564.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-565.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-566.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-567.json",
    "D:/Studying/gradu/013.피트니스자세/1.Training/라벨링데이터/맨몸운동_Labeling_new_220128/맨몸운동_10/Day36_201108_F/D36-1-568.json"
]

# ✅ 라벨 수정: 각 관절별로 0(정상), 1(잘못됨) 표시
labels = [
    [0]*24, [0]*24, [0]*24, [0]*24, [0]*24, [0]*24, [0]*24,   # 정상
    [1 if i in [20] else 0 for i in range(24)],  # 등이 잘못됨
    [1 if i in [7, 8] else 0 for i in range(24)], # 팔꿈치
    [1 if i in [5, 6] else 0 for i in range(24)], # 양쪽 어깨 = 가슴
    [1 if i in [17, 18] else 0 for i in range(24)], # 목과 허리가 잘못됨
    [1]*24,  # 모든 관절이 잘못된 경우 (예시)
]

# ✅ 전처리 실행
X_train, y_train = load_labeled_json_skeleton(file_paths, labels)

# ✅ 전처리된 데이터 형태 확인
print("전처리된 데이터 Shape:", X_train.shape)
print("전처리된 라벨 Shape:", y_train.shape)